In [5]:
import pandas as pd
import sys
import time

# Adicionando o caminho do diretório ao sys.path
sys.path.append(r"C:\Users\Daniel\Desktop\comparativo_frete")
from Scrap_minha_casa import executa_compara_frete_minha_casa_solar
from Scrap_neosolar import executa_compara_frete_neosolar
#from Scrap_energie_total import executa_compara_frete_energie_solar


In [6]:
# 33338 -painel 160W 
# 480 - Bomba Anauge P 100
# 13019- Bateria MOura
# 34405 - Painel Ztron 340W
# 31923  - Inversor Senoidal 1000W
product_ids_neo = ['480', '13019', '34405', '31923']
neo = executa_compara_frete_neosolar(product_ids_neo)

# 267779 -- Painel Resul 155W
# 265164 -- Bateria Moura
# 264790 -- Bomba P100
# 268645 -- Painel 340W Policristalino Half-Cell
# 269146 -- Iversor Senoidal 1000W - https://www.minhacasasolar.com.br/inversor-de-tensao-senoidal-1000w-12v-220v-hayonik-psw121-82646
product_ids_mcs = [265164, 264790, 268645, 269146]
mcs = executa_compara_frete_minha_casa_solar(product_ids_mcs)

            CEP  UF  Transportadora        Custo  Prazo    SKU Empresa
0     69940-000  AC     Econômico 1    R$ 153,93     20    480     Neo
1     69940-000  AC     Econômico 2    R$ 164,53     20    480     Neo
2     69940-000  AC    Correios PAC    R$ 230,30     31    480     Neo
3     69940-000  AC  Correios SEDEX    R$ 337,78     12    480     Neo
4     69940-000  AC        Rápido 1  R$ 2.859,73     15    480     Neo
...         ...  ..             ...          ...    ...    ...     ...
2503  77330-000  TO    Correios PAC    R$ 127,87     24  31923     Neo
2504  77330-000  TO     Econômico 2    R$ 133,46     14  31923     Neo
2505  77330-000  TO  Correios SEDEX    R$ 207,54     19  31923     Neo
2506  77330-000  TO     Econômico 3    R$ 236,03     22  31923     Neo
2507  77330-000  TO        Rápido 2    R$ 950,63     19  31923     Neo

[2508 rows x 7 columns]
Erro ao obter dados para o CEP: Status 502
Erro ao obter dados para o CEP: Status 502
Erro ao obter dados para o CEP: Statu

In [7]:
def trata_custo_mcs(df):
    df['SKU'] = df['SKU'].astype(str)
    df =  df[df['Transportadora'] != 'Retirada CD - Minha Casa Solar']

    df['Custo'] = df['Custo'].str.replace(',', '')
    df['Custo'] = df['Custo'].str.replace('.', '').astype(float)
    df['Custo'] = df['Custo']/100

    return df

def trata_custo_neo(df):
    df['SKU'] = df['SKU'].astype(str)
    df =  df[df['Transportadora'] != 'Retirada CD - Minha Casa Solar']

    df['Custo'] = df['Custo'].str.replace(r'R\$', '', regex=True)  # Remove 'R$'
    df['Custo'] = df['Custo'].str.replace(r'\xa0', '', regex=True)  # Remove '\xa0'
    df['Custo'] = df['Custo'].str.replace(',', '.')  # Substitui vírgula por ponto e converte para float
    df['Custo'] = df['Custo'].str.replace('.', '')
    df['Custo'] = df['Custo'].astype(float)
    df['Custo'] = df['Custo']/100

    return df

In [8]:
neo = trata_custo_neo(neo)
mcs = trata_custo_mcs(mcs)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_17608\58764820.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Custo'] = df['Custo'].str.replace(',', '')
C:\Users\Daniel\AppData\Local\Temp\ipykernel_17608\58764820.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Custo'] = df['Custo'].str.replace('.', '').astype(float)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_17608\58764820.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [27]:
neo.to_excel(rf"N:\Drives compartilhados\04 - OPERAÇÕES\70. Transporte\Acompanhamento_Entregas\Teste Unificação\DexPara_2024\Comparativo_bi_operacoes\Comparativo_frete_neo_{time.strftime('%Y-%m-%d')}.xlsx", index=False)
mcs.to_excel(rf"N:\Drives compartilhados\04 - OPERAÇÕES\70. Transporte\Acompanhamento_Entregas\Teste Unificação\DexPara_2024\Comparativo_bi_operacoes\Comparativo_frete_mcs_{time.strftime('%Y-%m-%d')}.xlsx", index=False)

In [10]:
neo.head(5)

,CEP,UF,Transportadora,Custo,Prazo,SKU,Empresa
0,69940-000,AC,Econômico 1,153.93,20,480,Neo
1,69940-000,AC,Econômico 2,164.53,20,480,Neo
2,69940-000,AC,Correios PAC,230.30,31,480,Neo
3,69940-000,AC,Correios SEDEX,337.78,12,480,Neo
4,69940-000,AC,Rápido 1,2859.73,15,480,Neo


In [11]:
def calculamedia(tes):
  menor_prazo_menor_custo = acha_menor_custo_menor_prazo(tes)

  media = menor_prazo_menor_custo.groupby(['SKU','Empresa','UF']).agg(media_preço = ('Custo', 'mean'), media_prazo = ('Prazo', 'mean')).reset_index()

  return media

def acha_menor_custo_menor_prazo(tes):

  # Agrupamos por 'UF' e 'SKU', e para cada grupo, pegamos a linha com o menor custo
  min_custo_prazo_do_custo = tes.loc[tes.groupby(['SKU','CEP','UF'])['Custo'].idxmin()]
  
  return min_custo_prazo_do_custo


In [12]:
dim_mcs = calculamedia(mcs)
dim_neo = calculamedia(neo)

In [13]:
def classifica_produto(df):
    # Listas de SKUs para comparação
    bomba_skus = ['480', '264790']
    bateria_skus = ['13019', '265164']
    Painel_155 = ['33338', '267779']
    Painel_340 = ['34405','268645'] 
    Inversor_Senoidal_100W = ['31923','269146']

    # Função para classificar o produto com base no SKU
    def classify_product(sku):
        if sku in bomba_skus:
            return 'Bomba Solar P100'
        elif sku in bateria_skus:
            return 'Bateria Moura 220aH'
        elif sku in Painel_155:
            return 'Painel 155W'
        elif sku in Painel_340:
            return 'Painel 340W Ztroon'
        elif sku in Inversor_Senoidal_100W:
            return 'Inversor Senoidal 1000W'

    # Aplicar a função para criar a nova coluna 'Produto'
    df['Produto'] = df['SKU'].apply(classify_product)

    return df

In [14]:
dim_neo = classifica_produto(dim_neo)
dim_mcs = classifica_produto(dim_mcs)

In [15]:
def calcular_diferenca_e_porcentagem(df1, df2):
    # Passo 1: Mesclar os DataFrames com base na coluna 'UF'
    merged_df = pd.merge(df1, df2, on='UF', how='inner')

    # Passo 2: Subtrair os valores das colunas 'media_preço_x' e 'media_preço_y'
    merged_df['Dif_Preço'] = merged_df['media_preço_x'] - merged_df['media_preço_y']
    merged_df['Dif_Prazo'] = merged_df['media_prazo_x'] - merged_df['media_prazo_y']

    # Passo 3: Calcular a diferença em porcentagem e formatar as colunas
    merged_df['%Dif_Preço'] = ((merged_df['Dif_Preço'] / merged_df['media_preço_x']) * 100).round(2)
    merged_df['%Dif_Prazo'] = ((merged_df['Dif_Prazo'] / merged_df['media_prazo_x']) * 100).round(2)
    #merged_df['%Dif_Preço'] = ((merged_df['Dif_Preço'] / merged_df['media_preço_x']) * 100).round(2).map("{:.2f}%".format)
    #merged_df['%Dif_Prazo'] = ((merged_df['Dif_Prazo'] / merged_df['media_prazo_x']) * 100).round(2).map("{:.2f}%".format)

    return merged_df

In [16]:
wdwdwd = pd.merge(dim_neo, dim_mcs, on=['Produto', 'UF'], how='inner')
wdwdwd.head(5)

,SKU_x,Empresa_x,UF,media_preço_x,media_prazo_x,Produto,SKU_y,Empresa_y,media_preço_y,media_prazo_y
0,13019,Neo,AC,338.192,25.8,Bateria Moura 220aH,265164,MCS,683.476,23.6
1,13019,Neo,AL,150.360,17.0,Bateria Moura 220aH,265164,MCS,227.774,14.0
2,13019,Neo,AM,545.138,42.2,Bateria Moura 220aH,265164,MCS,602.266,25.0
3,13019,Neo,AP,363.750,36.0,Bateria Moura 220aH,265164,MCS,338.900,18.0
4,13019,Neo,BA,202.214,24.6,Bateria Moura 220aH,265164,MCS,226.788,17.2


In [17]:
def calcular_diferenca_e_porcentagem(df1, df2):
    # Passo 1: Mesclar os DataFrames com base na coluna 'UF'
    merged_df = pd.merge(df1, df2, on=['Produto', 'UF'], how='inner')

    # Passo 2: Subtrair os valores das colunas 'media_preço_x' e 'media_preço_y'
    merged_df['Dif_Preço'] = merged_df['media_preço_x'] - merged_df['media_preço_y']
    merged_df['Dif_Prazo'] = merged_df['media_prazo_x'] - merged_df['media_prazo_y']

    # Passo 3: Calcular a diferença em porcentagem e formatar as colunas
    merged_df['%Dif_Preço'] = ((merged_df['Dif_Preço'] / merged_df['media_preço_x']) * 100).round(2)
    merged_df['%Dif_Prazo'] = ((merged_df['Dif_Prazo'] / merged_df['media_prazo_x']) * 100).round(2)
    #merged_df['%Dif_Preço'] = ((merged_df['Dif_Preço'] / merged_df['media_preço_x']) * 100).round(2).map("{:.2f}%".format)
    #merged_df['%Dif_Prazo'] = ((merged_df['Dif_Prazo'] / merged_df['media_prazo_x']) * 100).round(2).map("{:.2f}%".format)

    return merged_df

In [18]:
regioes = pd.read_excel(r"N:\Drives compartilhados\28 - Dados BI\2 Dados Transformados\Tabelas dimensão\D_Regiões e UF.xlsx")
regioes = regioes[['Região', 'UF']]
regioes.head(5)

,Região,UF
0,Norte,AC
1,Nordeste,AL
2,Norte,AM
3,Norte,AP
4,Nordeste,BA


In [19]:
comparativo_semi = calcular_diferenca_e_porcentagem(dim_neo,dim_mcs)
comparativo_semi = pd.merge(comparativo_semi, regioes, on=['UF'], how='inner')
comparativo_semi.head(5)

,SKU_x,Empresa_x,UF,media_preço_x,media_prazo_x,Produto,SKU_y,Empresa_y,media_preço_y,media_prazo_y,Dif_Preço,Dif_Prazo,%Dif_Preço,%Dif_Prazo,Região
0,13019,Neo,AC,338.192,25.8,Bateria Moura 220aH,265164,MCS,683.476,23.60,-345.284,2.20,-102.10,8.53,Norte
1,31923,Neo,AC,127.870,34.0,Inversor Senoidal 1000W,269146,MCS,188.000,13.50,-60.130,20.50,-47.02,60.29,Norte
2,34405,Neo,AC,168.986,28.0,Painel 340W Ztroon,268645,MCS,323.860,24.25,-154.874,3.75,-91.65,13.39,Norte
3,480,Neo,AC,169.204,23.8,Bomba Solar P100,264790,MCS,118.120,14.00,51.084,9.80,30.19,41.18,Norte
4,13019,Neo,AL,150.360,17.0,Bateria Moura 220aH,265164,MCS,227.774,14.00,-77.414,3.00,-51.49,17.65,Nordeste


In [20]:
# Corrigido para usar uma lista ao invés de uma tupla
comparativo = comparativo_semi.groupby(['Região', 'Produto'])[['%Dif_Preço', '%Dif_Prazo']].mean().reset_index()
comparativo = comparativo.sort_values(by='Produto', ascending=True)
comparativo['%Dif_Preço'] = comparativo['%Dif_Preço'].round(2)
comparativo['%Dif_Prazo'] = comparativo['%Dif_Prazo'].round(2)
comparativo

,Região,Produto,%Dif_Preço,%Dif_Prazo
0,Centro Oeste,Bateria Moura 220aH,10.52,-59.54
16,Sul,Bateria Moura 220aH,-17.58,-57.15
12,Sudeste,Bateria Moura 220aH,65.83,-35.64
4,Nordeste,Bateria Moura 220aH,-0.01,22.17
8,Norte,Bateria Moura 220aH,-46.35,10.19
20,São Paulo,Bateria Moura 220aH,28.29,-110.78
13,Sudeste,Bomba Solar P100,16.52,30.27
17,Sul,Bomba Solar P100,21.90,5.78
9,Norte,Bomba Solar P100,37.00,28.76
5,Nordeste,Bomba Solar P100,46.87,45.96


In [160]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

In [161]:
def style_and_round(df):
    # Arredondar as colunas %Dif_Preço e %Dif_Prazo para 2 casas decimais
    df['%Dif_Preço'] = df['%Dif_Preço'].round(2)
    df['%Dif_Prazo'] = df['%Dif_Prazo'].round(2)
    
    # Função para colorir as células com base nos valores
    def colorize(val):
        if val < 0:
            return 'background-color: green; color: white'
        else:
            return 'background-color: red; color: white'
    
    # Aplicar a função de formatação ao DataFrame
    return df.style.applymap(colorize, subset=['%Dif_Preço', '%Dif_Prazo'])


# Aplicar a função para arredondar e colorir
styled_comparativo = style_and_round(comparativo)

# Exibir o DataFrame estilizado
styled_comparativo

,Região,Produto,%Dif_Preço,%Dif_Prazo
0,Centro Oeste,Bateria Moura 220aH,2.260000,-50.340000
16,Sul,Bateria Moura 220aH,-16.880000,-54.530000
12,Sudeste,Bateria Moura 220aH,58.540000,-47.400000
4,Nordeste,Bateria Moura 220aH,-0.370000,23.050000
8,Norte,Bateria Moura 220aH,-46.550000,10.330000
20,São Paulo,Bateria Moura 220aH,28.660000,-105.880000
13,Sudeste,Bomba Solar P100,16.000000,29.520000
17,Sul,Bomba Solar P100,20.290000,1.400000
9,Norte,Bomba Solar P100,37.310000,29.740000
5,Nordeste,Bomba Solar P100,47.230000,46.440000


In [162]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Função para aplicar a coloração
def color_negative_red(val):
    color = 'red' if val > 0 else 'green'
    return f'color: {color}'

def color_negative_reds(df):
    # Aplica a coloração apenas nas colunas de interesse
    return df.style.applymap(color_negative_red, subset=['%Dif_Preço', '%Dif_Prazo'])

# Exibindo a tabela estilizada
styled_df = color_negative_reds(comparativo)
styled_df


,Região,Produto,%Dif_Preço,%Dif_Prazo
0,Centro Oeste,Bateria Moura 220aH,2.260000,-50.340000
16,Sul,Bateria Moura 220aH,-16.880000,-54.530000
12,Sudeste,Bateria Moura 220aH,58.540000,-47.400000
4,Nordeste,Bateria Moura 220aH,-0.370000,23.050000
8,Norte,Bateria Moura 220aH,-46.550000,10.330000
20,São Paulo,Bateria Moura 220aH,28.660000,-105.880000
13,Sudeste,Bomba Solar P100,16.000000,29.520000
17,Sul,Bomba Solar P100,20.290000,1.400000
9,Norte,Bomba Solar P100,37.310000,29.740000
5,Nordeste,Bomba Solar P100,47.230000,46.440000
